# CNNs pentru clasificarea textelor

Folosirea CNN-urilor pentru a clasifica textul a fost prima dată prezentată în: [Convolutional Neural Networks for Sentence Classification](https://aclanthology.org/D14-1181.pdf).

### Arhitectura CNN

<img src="https://richliao.github.io/images/YoonKim_ConvtextClassifier.png">

Dându-se ca input un text de $n$ cuvinte $w_{1}$, $w_{2}$, ..., $w_{n}$, transformăm fiecare cuvânt într-un vector de dimensiune $d$, rezultând vectorii $w_{1}$, $w_{2}$, ..., $w_{n}$ aparținând $R^d$. Matricea rezultată de dimensiune $d$×$n$ este apoi folosită ca input pentru un layer convoluțional care trece un *sliding window* peste text.

Pentru fiecare window de lungime $l$:

$u_{i}$ = [$w_{i}$, ..., $w_{i+l-1}$] $∈ R^{d×l}$, 0≤$i$≤$n-l$ 

Pentru fiecare filtru $f_{j} ∈ R^{d×l}$ calculăm <$u_{i}$, $f_{j}$> și obținem matricea $ F ∈ R^{m×n}$ (dacă am făcut padding înainte de aplicarea filtrului, astfel încât să păstrăm dimensiunea $n$ a cuvintelor), unde $m$ este numărul de filtre. Aplicăm max-pooling pe matricea $F$ rezultată, apoi aplicăm funcția de activare. În final, avem un layer *fully connected* care produce distribuția pe clase, din care rezultă clasa cu cea mai mare probabilitate.

### Convoluții și filtre

<img src="https://debajyotidatta.github.io/assets/images/conv.001.png" width="300">

**Inputul** este format dintr-o matrice de dimensiune $n$×$d$, unde $n$ este numărul de cuvinte sau caractere, iar $d$ este lungimea reprezentării vectoriale sau lungimea vocabularului.  

De exemplu, pentru reprezentarea vectorială a unui text la nivel de caracter, pentru $d$ = 70, numărul de caractere unice în vocabular, pe fiecare linie a matricei avem reprezentarea one-hot a unui caracter.

<img src="https://debajyotidatta.github.io/assets/images/conv.002.png" width="500">

**Filtrele** (kernels) pot avea orice lungime. Lungimea este dată de numărul de linii din filtru. Lățimea filtrului trebuie să fie aceeași cu numărul de coloane din reprezentarea vectorială ($d$).

<img src="https://debajyotidatta.github.io/assets/images/conv.003.png" width="300">

Operația de convoluție presupune multiplicarea elementelor din input și filtru, rezultând o valoare care reprezintă suma rezultatelor multiplicării. În consecință, operația de convoluție multiplică *weight*-urile din filtru cu reprezentarea vectorială a cuvintelor.

<img src="https://debajyotidatta.github.io/assets/images/conv.004.png" width="300">

<img src="https://debajyotidatta.github.io/assets/images/conv.005.png" width="300">

<img src="https://debajyotidatta.github.io/assets/images/conv.006.png" width="300">

Filtrul este aplicat secvențial peste input, dar, la fel ca în cazul imaginilor, putem folosi diferite valori pentru *stride* pentru a controla cât de mult de mișcă filtrul vertical. Utilizând *stride* cu o valoare $k$ putem aplica un filtrul din $k$ în $k$ linii. De exemplu, pentru un filtru cu stride = 2, filtrul va fi aplicat pe secvența de text din 2 în 2 linii și vom avea un output de dimensiune mai mică.

<img src="https://debajyotidatta.github.io/assets/images/conv2.006.png" width="450">

Filtre multiple vor produce output-uri multiple. 

<img src="https://debajyotidatta.github.io/assets/images/conv2.007.png" width="450">

La următorul pas se realizează *max pooling* peste fiecare feature map rezultat din aplicarea filtrelor, iar apoi rezultatele sunt concatenate. 



### Imagini vs Text

Pentru a înțelege de ce o abordare folosind CNN-uri este potrivită pentru text, trebuie să ne gândim la textele noastre ca fiind niște imagini.

Pentru exemplul următor vom cosidera că reprezentarea unei propoziții a fost făcută la nivel de cuvânt.

De exemplu, pentru o propoziție cu lungimea maximă de 70 de cuvinte și lungimea embeddingului egală cu 300, putem crea o matrice cu valori numerice de forma 70x300 pentru a reprezenta această propoziție. Spre deosebire de imagini, în care elementele matricei sunt reprezentate de valori ale pixelilor, fiecare linie din reprezentarea vectorială a propoziției este, de fapt, reprezentarea unui cuvânt.

În cazul imaginilor, filtrul de convoluție se deplasează și vertical și orizontal, dar în cazul textului, filtrul se deplasează doar vertical, convoluțiile sunt doar 1D. Un kernel de dimensiune (2, 300), care are dimensiunea filtrului egală cu 2 se uită doar la 2 cuvinte în același timp. Ne putem gândi, deci, la dimensionea filtrelor ca la o dimensiune a n-gramelor (bigrame, trigrame, etc.).

În acest laborator vom folosi datasetul IMDb movie reviews: https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [70]:
! pip install unidecode

In [1]:
import torch
import pandas as pd
from pprint import pprint
from sklearn.model_selection import train_test_split
from unidecode import unidecode
from collections import Counter
import nltk
from nltk import word_tokenize
nltk.download('punkt')


[nltk_data] Downloading package punkt to C:\Users\Florentin-Giuliano
[nltk_data]     D\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from urllib.request import urlretrieve
urlretrieve('https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv', 'IMDB_Dataset.csv')

('IMDB_Dataset.csv', <http.client.HTTPMessage at 0x1e231b222e0>)

In [3]:
data = pd.read_csv('IMDB_Dataset.csv')
data = data[:10000]
data

,review,sentiment
0,My family and I normally do not watch local mo...,1
1,"Believe it or not, this was at one time the wo...",0
2,"After some internet surfing, I found the ""Home...",0
3,One of the most unheralded great works of anim...,1
4,"It was the Sixties, and anyone with long hair ...",0
...,...,...
9995,"The film maybe goes a little far, but if you l...",1
9996,This two-parter was excellent - the best since...,1
9997,Shaggy & Scooby-Doo Get a Clue. It's like watc...,0
9998,"Todd Rohal is a mad genius. ""Knuckleface Jones...",1


Impartim datasetul in train si test.

In [4]:
train_df, test_df = train_test_split(data, test_size=0.20, random_state = 42)

print('Dimensiunea datelor de train', len(train_df))
print('Dimensiunea datelor de test', len(test_df))

Dimensiunea datelor de train 8000
Dimensiunea datelor de test 2000


Așa cum am văzut în laboratoarele trecute, nu putem antrena un model direct pe datele sub formă de text, trebuie să transformam datele în reprezentări numerice vectoriale. 

Pentru asta, trebuie să parcurgem 2 pași:

- **Tokenizare**: împărțirea textelor în subtexte mai mici. Astfel vom determina vocabularul setului nostru de date (setul de tokeni unici)

- **Vectorizare**: reprezentarea în format numeric vectorial

Textul poate fi reprezentat fie ca o secvență de caractere, fie ca o secvență de cuvinte. Utilizarea reprezentării la nivel de cuvânt are o performanță mai bună și este mai folosită, pe când reprezentarea la nivel de caracter este utilă dacă textele au multe greșeli de scriere. 

### Reprezentarea vectorială la nivel de caracter


```
Texts: 'the mouse ran up the clock' and 'the mouse ran down'
```

Pe langa caracterele prezente in textele noastre, adaugam si 2 tokeni speciali: UNK (unknown) si PAD.


```
Index assigned for every token: {0: 'UNK', 1: 'PAD', 2: 't', 3: 'm', 4: 'c', 5: 'h', 6: 'l', 7: 'w', 8: ' ', 9: 'a', 10: 'k', 11: 'e', 12: 'r', 13: 'u', 14: 'n', 15: 's', 16: 'd', 17: 'p', 18: 'o'}
```

Reprezentarea vectoriala a celor doua texte folosind indexul corespunzator pentru fiecare caracter:

```
'the mouse ran up the clock' = [2, 5, 11, 8, 3, 18, 13, 15, 11, 8, 12, 9, 14, 8, 13, 17, 8, 2, 5, 11, 8, 4, 6, 18, 4, 10]
'the mouse ran down' = [2, 5, 11, 8, 3, 18, 13, 15, 11, 8, 12, 9, 14, 8, 16, 18, 7, 14]
```

Adaugam valori de padding la cel de-al doilea vector pentru a avea o lungime egala cu primul vector si obtinem:

```
[2, 5, 11, 8, 3, 18, 13, 15, 11, 8, 12, 9, 14, 8, 16, 18, 7, 14, 1, 1, 1, 1, 1, 1, 1, 1]
```

Transformăm lista de review-uri într-o listă de caractere pentru fiecare review.

In [5]:
def transform_to_char(data):

    reviews = []
    
    for review in data:
        review_cleaned = [char.lower() for char in review]
        reviews.append(review_cleaned)

    return reviews

train_reviews = transform_to_char(train_df.review)
train_labels = train_df.sentiment.tolist()

print('Reviews', len(train_reviews))
print('Labels', len(train_labels))

Reviews 8000
Labels 8000


Calculam marimea vocabularului de caractere.

In [6]:
def get_vocab(data):

    units = set([unit for review in data for unit in review])
    
    return units

vocab = get_vocab(train_reviews)

print('total chars:', len(vocab))
print(vocab)

total chars: 128
{'½', '9', '“', '?', '\xa0', "'", '%', '«', '¢', '\x84', 'v', 'ñ', '\x97', 'n', 's', 'ß', 'ô', 'é', '\x96', '®', '¨', 'x', 'k', 'h', 'כ', 'c', '-', 'ü', ';', 'מ', '\x95', '@', '0', '.', '\x85', '=', 'á', 'q', 'ã', '’', '(', '»', 'j', 'b', 'f', 'a', 'y', '\x91', '\\', 'î', 'ó', 'ý', ']', '1', '^', 'l', 'i', '}', 'í', 'g', '_', 'u', '+', '$', ',', '\t', 'z', ':', 'à', '>', 't', '5', 'א', 'ð', '~', 'ן', 'w', '³', 'ä', '7', '´', '&', 'æ', 'ç', '[', '!', 'י', 'ל', '8', '/', 'ו', '…', 'ר', '‘', '–', '3', 'd', 'ë', ')', 'm', '£', 'ï', 'ג', '<', 'o', 'r', 'â', '"', '`', '2', 'å', '|', 'ö', 'õ', 'e', 'è', '6', '*', '”', '{', 'ê', 'ø', ' ', '4', '#', 'ú', 'p', '\x80'}


Putem vedea ca avem foarte multe caractere cu accente, diferite de caracterele limbii engleze. Pentru a micsora lungimea vocabularului, putem transforma caracterele utf8 in cea mai apropiata forma ASCII a lor.

In [7]:
reviews_to_ascii = [unidecode(review) for review in train_df.review]
train_reviews = transform_to_char(reviews_to_ascii)
vocab = get_vocab(train_reviews)

print('total chars:', len(vocab))
print(vocab)

total chars: 70
{',', '\t', 'z', '0', '.', ':', '=', 'q', '9', '<', 'o', '?', "'", 't', '>', 'r', '(', '5', '%', '~', '"', 'v', 'j', 'w', 'b', '`', 'f', '2', 'a', 'y', '\\', '|', '7', 'n', 's', ']', '1', 'e', '-', '&', '[', '!', '6', '^', 'l', '/', 'i', '8', '*', '}', '{', 'x', ';', 'k', 'g', 'h', ' ', '4', '3', '#', '_', 'd', 'p', 'u', 'c', ')', 'm', '+', '@', '$'}


Atribuim fiecarui caracter din vocabularul nostru un index. Vom atribui 0 caracterelor necunoscute, iar 1 va fi valoarea atribuita paddingului.

In [8]:
char_indices = dict((c, i + 2) for i, c in enumerate(vocab))
indices_char = dict((i + 2, c) for i, c in enumerate(vocab))

indices_char[0] = 'UNK'
char_indices['UNK'] = 0

indices_char[1] = 'PAD'
char_indices['PAD'] = 1

print('Dimensiunea vocabularului', len(indices_char))
print(indices_char)

Dimensiunea vocabularului 72
{2: ',', 3: '\t', 4: 'z', 5: '0', 6: '.', 7: ':', 8: '=', 9: 'q', 10: '9', 11: '<', 12: 'o', 13: '?', 14: "'", 15: 't', 16: '>', 17: 'r', 18: '(', 19: '5', 20: '%', 21: '~', 22: '"', 23: 'v', 24: 'j', 25: 'w', 26: 'b', 27: '`', 28: 'f', 29: '2', 30: 'a', 31: 'y', 32: '\\', 33: '|', 34: '7', 35: 'n', 36: 's', 37: ']', 38: '1', 39: 'e', 40: '-', 41: '&', 42: '[', 43: '!', 44: '6', 45: '^', 46: 'l', 47: '/', 48: 'i', 49: '8', 50: '*', 51: '}', 52: '{', 53: 'x', 54: ';', 55: 'k', 56: 'g', 57: 'h', 58: ' ', 59: '4', 60: '3', 61: '#', 62: '_', 63: 'd', 64: 'p', 65: 'u', 66: 'c', 67: ')', 68: 'm', 69: '+', 70: '@', 71: '$', 0: 'UNK', 1: 'PAD'}


Acum putem transforma propozitiile din datasetul nostru intr-o reprezentare vectoriala, in care vom avea pentru fiecare caracter indicele corespunzator din vocabularul nostru.

In [9]:
import numpy as np

def vectorize_sentences(data, char_indices, one_hot = False):
    vectorized = []
    for sentences in data:

        # transformam fiecare review in reprezentarea lui sub forma de indici ale caracterelor continute
        sentences_of_indices = [char_indices[w] if w in char_indices.keys() else char_indices['UNK'] for w in sentences]

        # pentru fiecare indice putem face reprezentarea one-hot corespunzatoare
        # sau putem sa nu facem asta si sa adaugam un embedding layer in model care face această transformare
        if one_hot:
            sentences_of_indices = np.eye(len(char_indices))[sentences_of_indices]

        vectorized.append(sentences_of_indices)

    return vectorized

train_reviews_vectorized = vectorize_sentences(train_reviews, char_indices)
# train_reviews_vectorized[0]

In [10]:
vectors_dim = [len(repr) for repr in train_reviews_vectorized]
vectors_dim[:5]

[1957, 370, 903, 3001, 1403]

Putem vedea ca deoarece review-urile au numar diferit de caractere, in consecinta, si dimensiunile reprezentarilor vectoriale sunt diferite. 
Vom aduce reprezentarile noastre la aceeasi dimensiune maxima.

Definim o functie *pad* care:

- primeste un set de review-uri si o lungime maxima
- scurteaza toate reprezentarile mai mari decat lungimea maxima
- adauga valoarea de padding (1 in cazul nostru) la reprezentarile mai scurte decat lungimea maxima

In [11]:
def pad(samples, max_length):
    
    return torch.tensor([
        sample[:max_length] + [1] * max(0, max_length - len(sample))
        for sample in samples
    ])

In [12]:
train_reviews_vectorized = pad(train_reviews_vectorized, max_length = 1000)
train_reviews_vectorized

tensor([[48, 15, 58,  ..., 65, 36, 15],
        [63, 48, 63,  ...,  1,  1,  1],
        [15, 57, 48,  ...,  1,  1,  1],
        ...,
        [48, 58, 46,  ...,  1,  1,  1],
        [63, 39, 36,  ...,  1,  1,  1],
        [15, 57, 48,  ...,  1,  1,  1]])

In [13]:
train_reviews_vectorized.shape

torch.Size([8000, 1000])

Tranformăm și review-urile din setul de test într-o reprezentare vectorială

In [14]:
test_reviews_to_ascii = [unidecode(review) for review in test_df.review]
test_reviews = transform_to_char(test_reviews_to_ascii)
test_labels = test_df.sentiment.tolist()

print('Reviews', len(test_reviews))
print('Labels', len(test_labels))

Reviews 2000
Labels 2000


In [15]:
test_reviews_vectorized = vectorize_sentences(test_reviews, char_indices)
test_reviews_vectorized = pad(test_reviews_vectorized, max_length = 1000)
# test_reviews_vectorized[0]

Vom incarca seturile noastre de date intr-un obiect din clasa [Dataset](https://pytorch.org/docs/stable/data.html?highlight=dataset#torch.utils.data.Dataset).

In [16]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, samples, labels):
        self.samples = samples
        self.labels = labels
            
    def __getitem__(self, k):
        """Returneaza al k-lea exemplu din dataset"""
        return self.samples[k], self.labels[k]
    
    def __len__(self):
        """Returneaza dimensiunea datasetului"""
        return len(self.samples)

Definim arhitectura modelului

In [17]:
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        # Definim un embedding layer cu un vocabular de dimensiune 72
        # și ca output un embedding de dimensiune 20
        # padding_idx este indexul din vocabular al paddingului (1, în cazul nostru)
        
        self.embedding = torch.nn.Embedding(72, 20, padding_idx=1)

        # Definim o secvență de layere
        
        # Un layer Convolutional 1D cu 20 input channels, 32 output channels, dimensiune kernel = 3 și padding = 1
        # ReLU activation
        # 1D Maxpooling layer de dimensiune 2
        conv1 = torch.nn.Sequential(
            torch.nn.Conv1d(in_channels=20, out_channels=32, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool1d(kernel_size=2),
        )
        
        # Un layer Convolutional 1D cu 32 input channels, 32 output channels, dimensiune kernel = 5 și padding = 2
        # ReLU activation
        # 1D Maxpooling layer de dimensiune 2
        conv2 = torch.nn.Sequential(
            torch.nn.Conv1d(in_channels=32, out_channels=32, kernel_size=5, padding=2),
            torch.nn.ReLU(),
            torch.nn.MaxPool1d(kernel_size=2),
        )
        
        # Global Average pooling layer care, în cazul nostru, este un 1D Avgerage Pooling layer
        # cu dimensiunea de 250 și stride 250
        global_average = torch.nn.AvgPool1d(kernel_size=250, stride=250)

        self.convolutions = torch.nn.Sequential(
            conv1, conv2, global_average
        )
        
        # Flattening layer
        flatten = torch.nn.Flatten()
        
        # Linear layer cu 32 input features și 2 outputs fără funcție de activare
        linear = torch.nn.Linear(in_features=32, out_features=2)

        self.classifier = torch.nn.Sequential(flatten, linear)
        
    def forward(self, input):
        # trecem inputul prin layerul de embedding
        embeddings = self.embedding(input)
        
        # permutăm inputul astfel încât prima dimensiune este numărul de channels
        embeddings = embeddings.permute(0, 2, 1)
        
        # trecem inputul prin secvența de layere
        output = self.convolutions(embeddings)
        output = self.classifier(output)
        return output


In [18]:
DEVICE = torch.device("cuda")
# instanțiem modelul
model = Model().to(DEVICE)

# Adam optimizer cu lr = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Cross Entropy loss
loss_fn = torch.nn.CrossEntropyLoss()

# training dataset and dataloader
# test dataset and dataloader
train_ds = Dataset(train_reviews_vectorized, train_labels)
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=64, shuffle=True)
test_ds = Dataset(test_reviews_vectorized, test_labels)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=64, shuffle=False)

AssertionError: Torch not compiled with CUDA enabled

Training loop

In [ ]:
best_val_acc = 0
for epoch_n in range(10):
    print(f"Epoch #{epoch_n + 1}")
    model.train()
    for batch in train_dl:
        model.zero_grad()

        inputs, targets = batch
        inputs = inputs.long().to(DEVICE)
        targets = targets.to(DEVICE)

        output = model(inputs)
        loss = loss_fn(output, targets)

        loss.backward()
        optimizer.step()

    # validare
    model.eval()
    all_predictions = torch.tensor([])
    all_targets = torch.tensor([])
    for batch in test_dl:
        inputs, targets = batch
        inputs = inputs.long().to(DEVICE)
        targets = targets.to(DEVICE)

        with torch.no_grad():
            output = model(inputs)

        predictions = output.argmax(1)
        all_targets = torch.cat([all_targets, targets.detach().cpu()])
        all_predictions = torch.cat([all_predictions, predictions.detach().cpu()])

    val_acc = (all_predictions == all_targets).float().mean().numpy()
    print(val_acc)

    if val_acc > best_val_acc:
        torch.save(model.state_dict(), "./model")
        best_val_acc = val_acc

print("Best validation accuracy", best_val_acc)

Epoch #1
0.5185
Epoch #2
0.527
Epoch #3
0.623
Epoch #4
0.637
Epoch #5
0.625
Epoch #6
0.6095
Epoch #7
0.6675
Epoch #8
0.658
Epoch #9
0.657
Epoch #10
0.6855
Best validation accuracy 0.6855


### Reprezentarea vectoriala la nivel de cuvant



```
Texts: 'The mouse ran up the clock' and 'The mouse ran down'
```

Pe langa tokenii prezenti in textele noastre, adaugam si 2 tokeni speciali: UNK (unknown word) si PAD.


```
Index assigned for every token: {'UNK': 0, 'PAD': 1, 'the': 2, 'mouse': 3, 'ran': 4, 'up': 5, 'clock': 6, 'down': 7}
```

Reprezentarea vectoriala a celor doua texte folosind indexul corespunzator pentru fiecare cuvant:

```
'The mouse ran up the clock' = [2, 3, 4, 5, 2, 6]
'The mouse ran down' = [2, 3, 4, 7]
```

Adaugam valori de padding la cel de-al doilea vector pentru a avea o lungime egala cu primul vector si obtinem:

```
[2, 3, 4, 7, 1, 1]
```
Reprezentarea one-hot a fiecarui text:

```
'The mouse ran up the clock' = [[0. 0. 1. 0. 0. 0. 0.]
                                [0. 0. 0. 1. 0. 0. 0.]
                                [0. 0. 0. 0. 1. 0. 0.]
                                [0. 0. 0. 0. 0. 1. 0.]
                                [0. 0. 1. 0. 0. 0. 0.]
                                [0. 0. 0. 0. 0. 0. 1.]]

'The mouse ran down' = [[0. 0. 1. 0. 0. 0. 0. 0.]
                        [0. 0. 0. 1. 0. 0. 0. 0.]
                        [0. 0. 0. 0. 1. 0. 0. 0.]
                        [0. 0. 0. 0. 0. 0. 0. 1.]
                        [0. 1. 0. 0. 0. 0. 0. 0.]]
```

Impartim textele din dataset in tokeni.

In [ ]:
def transform_to_tokens(data):

    reviews = []
    for review in data:
        review_tokenized = word_tokenize(review.lower())
        reviews.append(review_tokenized)

    return reviews

In [ ]:
train_reviews = transform_to_tokens(train_df.review)
for r in train_reviews[:2]:
    print(r[:20])

['it', 'is', 'interesting', 'to', 'see', 'what', 'people', 'think', 'of', 'this', 'movie', ',', 'since', 'it', 'is', ',', 'in', 'fact', ',', 'quite']
['did', 'the', 'first', 'travesty', 'actually', 'make', 'money', '?', 'this', 'is', 'another', 'sequel', '(', 'along', 'the', 'lines', 'of', 'another', 'stakeout', ')']


Construim vocabularul de tokeni

In [ ]:
vocab = get_vocab(train_reviews)

print('total words:', len(vocab))
print(list(vocab)[:100])

total words: 64266
['dr.tadokoro', 'jibes', 'ooooohhhh', 'advertised', 'willam', 'isolationist', 'redd', 'five-hundred-million', 'horsing', 'absurdest', 'gifts.', 'practitioner', 'tackles', 'turtlenecks', 'malta', '-screened', 'movie/theatre', 'drank', 'weaned', 'explicit', 'alluring.', 'renewal', 'cronenbergs', 'perplexity', 'sensibilities.', 'heck-of-a', 'shearer', 'received-', 'tastes', 'gurning', 'giardello', 'body-dissolving', 'dvd.', 'rewrote', 'bos', 'sharpen', 'picerni', 'context.', 'crushes', 'grand-mal', 'meet.so', 'ooooo', 'thriller.', 'barter', 'odder', 'raciest', 'amounted', 'no-talent', '3.99', 'elo', 'oppenheimer.', 'havegotten', 'trashes', 'hasidic', 'soup', 'inter', 'pappy', 'friel', 'missing.', 'reviewers', 'parole', 'c-', 'accentuated', 'doormen', 'pfieffer', 'riverbank.', 'piggy', 'brigite', 'flynn/gable', 'dilo', 'nationality', 'unfunniness', 'acting-directing', 'strong-willed', 'good/bad', 'almost-parallels', 'minnesota', 'fully-realized', 'me', 'muzaffer', 'stage

Avem un vocabular foarte mare. Vom scoate din vocabular cuvintele cu o frecventa foarte mica. 

O alta abordare pentru micscorarea vocabularului este scoaterea cuvintelor foarte frecvente (stopwords). De asemenea putem face si alti pasi de preprocesare: putem face stemming, lematizare, putem scoate punctuatia, etc.)

In [ ]:
import operator

def word_freq(data, min_aparitions):
    
    all_words = [words.lower() for sentences in data for words in sentences]
    sorted_vocab = sorted(dict(Counter(all_words)).items(), key=operator.itemgetter(1))
    final_vocab = [k for k,v in sorted_vocab if v > min_aparitions]

    return final_vocab

In [ ]:
vocab = word_freq(train_reviews, min_aparitions = 10)

print(vocab[:100])
print(len(vocab))

['toll', 'denying', 'reported', 'shaft', 'histrionics', 'questioned', 'hurting', 'deliverance', 'backwoods', 'translate', 'dreadfully', 'bump', 'caper', 'sophomoric', 'hypnotic', 'washed', 'playful', 'stream', 'baddie', 'humiliating', 'necessity', 'incompetence', 'amidst', 'exposes', 'sirk', 'marc', 'interspersed', 'paste', 'grieving', 'tightly', 'masked', 'boil', 'marvin', 'cliffhanger', 'airing', 'video.', 'hilarious.', 'schneider', 'covert', 'holland', 'somber', 'salvage', 'sydow', 'towering', 'cannibalism', 'knives', 'shepard', 'remainder', 'two-dimensional', 'virtues', 'adapting', 'incidental', "'it", '..i', 'sites', 'observed', 'exploiting', 'ploy', 'czech', 'conflicted', 'stares', 'chiller', 'monument', 'raj', 'progressively', 'jox', 'tournament', 'stir', 'atop', 'shelter', 'backing', 'dives', 'dangers', 'unleashed', 'atomic', 'build-up', 'b-grade', 'garland', 'hamming', 'steele', 'orchestral', 'switched', 'opponent', 'cunning', 'chef', 'lottery', 'sumptuous', 'potent', 'spinnin

Fiecarui token ii atribuim un indice.

In [ ]:
word_indices = dict((c, i + 2) for i, c in enumerate(vocab))
indices_word = dict((i + 2, c) for i, c in enumerate(vocab))

indices_word[0] = 'UNK'
word_indices['UNK'] = 0

indices_word[1] = 'PAD'
word_indices['PAD'] = 1

In [ ]:
# print(indices_word)

Acum putem transforma propozitiile din datasetul nostru intr-o reprezentare vectoriala, in care vom avea pentru fiecare cuvant indicele corespunzator din vocabular.

In [ ]:
train_reviews_vectorized = vectorize_sentences(train_reviews, word_indices)

In [ ]:
vectors_dim = [len(repr) for repr in train_reviews_vectorized]
vectors_dim[:5]

[423, 77, 186, 580, 288]

Din nou, pentru ca textele au un numar diferit de cuvinte, trebuie sa aducem vectorii la o reprezentare de aceeași dimensiune.

In [ ]:
train_reviews_vectorized = pad(train_reviews_vectorized, max_length = 512)
train_reviews_vectorized

tensor([[9707, 9712, 9486,  ...,    1,    1,    1],
        [9635, 9719, 9619,  ...,    1,    1,    1],
        [9704, 9697, 9712,  ...,    1,    1,    1],
        ...,
        [9705, 9301, 9719,  ...,    1,    1,    1],
        [9261, 9719,  251,  ...,    1,    1,    1],
        [9704, 9712, 9567,  ...,    1,    1,    1]])

In [ ]:
train_reviews_vectorized.shape

torch.Size([8000, 512])

# TASK
## Deadline: 31 martie ora 23:59.

Formular pentru trimiterea temei: https://forms.gle/Bznaciv2MTy4kVL47

Folosind intreg datasetul de mai sus (IMDb reviews) implementati urmatoarele cerinte:
1. Impartiti setul de date in 80% train, 10% validare si 10% test
2. Tokenizati textele si determinati vocabularul (in acest task vom lucra cu reprezentari la nivel de cuvant, NU la nivel de caracter); intrucat vocabularul poate fi foarte mare, incercati sa aplicati una dintre tehnicile mentionate in laborator (10K-20K de cuvinte ar fi o dimensiunea rezonabila a vocabularului)
3. Transformati textele in vectori de aceeasi dimensiune folosind indexul vocabularului (alegeti o dimensiune maxima de circa 500-1000 de tokens)
4. Implementati urmatoarea arhitectura:
    * un Embedding layer pentru vocabularul determinat, ce contine vectori de dimensiune 100
    * un layer dropout cu probabilitate 0.4
    * un layer convolutional 1D cu 100 canale de input si 128 de canale de output, dimensiunea kernelului de 3 si padding 1; asupra rezultatului aplicati un layer de [BatchNormalization](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html) cu 128 features; aplicati apoi functia de activare ReLU, iar in cele din urma un strat de max-pooling 1D cu kernel size 2.
    * un layer convolutional 1D cu 128 canale de input si 128 de canale de output, dimensiunea kernelului de 5 si padding 2; asupra rezultatului aplicati un layer de BatchNormalization cu 128 features; aplicati apoi functia de activare ReLU, iar in cele din urma un strat de max-pooling 1D cu kernel size 2.
    * un layer convolutional 1D cu 128 canale de input si 128 de canale de output, dimensiunea kernelului de 5 si padding 2; asupra rezultatului aplicati un layer de BatchNormalization cu 128 features; aplicati apoi functia de activare ReLU, iar in cele din urma un strat de max-pooling 1D cu kernel size 2.
    * asupra rezultatului ultimului layer, aplicati average-pooling 1D obtinand pentru fiecare canal media tuturor valorilor din vectorul sau corespunzator
    * un layer feed-forward (linear) cu dimensiunea inputului 128, si 2 noduri pentru output (pentru clasificare in 0/1)
5. Antrenati arhitectura folosind cross-entropy ca functie de loss si un optimizer la alegere. La finalul fiecarei epoci evaluati modelul pe datele de validare si salvati weighturile celui mai bun model astfel determinat
6. Evaluati cel mai bun model obtinut pe datele de test.


In [1]:
# 0 Partea de definire a librariilor

import torch
import pandas as pd
from pprint import pprint
from sklearn.model_selection import train_test_split
from unidecode import unidecode
from collections import Counter
import nltk
from nltk import word_tokenize
from urllib.request import urlretrieve

nltk.download('punkt')
urlretrieve('https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv', 'IMDB_Dataset.csv')

[nltk_data] Downloading package punkt to C:\Users\Florentin-Giuliano
[nltk_data]     D\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


('IMDB_Dataset.csv', <http.client.HTTPMessage at 0x2204da351f0>)

In [11]:
#1 

data = pd.read_csv('IMDB_Dataset.csv')

texts = data['review']
labels = data['sentiment']

x_train, x_aux, y_train, y_aux = train_test_split(texts, labels, test_size=0.20, random_state = 42)
x_test,x_valid, y_test, y_valid = train_test_split(x_aux,y_aux,test_size=0.50, random_state = 42)

print(len(y_train))

40000


In [12]:
'''
    ex 2
    Vom face tokenizarea si urmatoarele operatii, scot stop word urile, scot cuvintele cu frecventa mica, scot punctuatia, aplic si stemming
'''
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('english')
stopwords_list = set(stopwords.words('english'))

def processing(data):
    data.lower()
    data = re.sub('[^a-z]', ' ', data)
    return data

def tokenize(data):
    data = word_tokenize(data)
    data = [i for i in data if not i in stopwords_list]
    return  [stemmer.stem(i) for i in data]

vocDic = {}

maxWords = 10000

def processData(data,makeDic = True):
    newData = []
    n = 0
    for i in data:
        text = tokenize(processing(i))
        newData.append(text)
        if makeDic:
            for j in text:
                if not vocDic.get(j) and n < maxWords:
                    vocDic[j] = n
                    n += 1
                if n == maxWords:
                    break
    return newData

x_test = processData(x_test, False)
x_train = processData(x_train, True)
x_valid = processData(x_valid,False)

In [13]:
# 3
MAX_LENGTH = 1000

def vectorized_sentences(data): # List[List[word]]
    newData = []
    for sentence in data:
        vector = []
        for word in sentence:
            index = 0
            if vocDic.get(word):
                index = vocDic[word]
            vector.append(index)
        vector = vector[:MAX_LENGTH]
        n = MAX_LENGTH - len(vector)
        vector.extend(1 for i in range(n))
        newData.append(vector)
    return newData

x_train = vectorized_sentences(x_train)
x_test = vectorized_sentences(x_test)
x_valid = vectorized_sentences(x_valid)

In [14]:
# 3 -- verificare

print(x_train[0])


# facem dataseturile
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        super().__init__()
        self.data = data
        
    def __getitem__(self, k):
        return (
            self.data["x"][k],
            self.data["y"][k]
        )
    
    def __len__(self):
        return len(self.data["y"])

def generateDataloader(x, y):
    x = torch.tensor(x)
    y = torch.tensor(y.tolist())
    data = {
        'x': x,
        'y': y
    }
    dataset = MyDataset(data)
    return torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)

datasetTrain = generateDataloader(x_train,y_train)
datasetTest = generateDataloader(x_test,y_test)
datasetValidate = generateDataloader(x_valid,y_valid)

[2669, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 5, 14, 15, 16, 6, 17, 18, 19, 7, 20, 8, 21, 22, 23, 24, 25, 13, 26, 27, 5, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 18, 19, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 1, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 60, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [15]:
# 4

class CNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.emb = torch.nn.Embedding(maxWords,100,padding_idx = 1)
        self.model = torch.nn.Sequential(
            torch.nn.Dropout(0.4),
            torch.nn.Conv1d(in_channels  = 100,out_channels  = 128,kernel_size  = 3,padding = 1),
            torch.nn.BatchNorm1d(128),
            torch.nn.ReLU(),
            torch.nn.MaxPool1d(kernel_size=2),
            torch.nn.Conv1d(in_channels  =128,out_channels =128,kernel_size =5,padding = 2),
            torch.nn.BatchNorm1d(128),
            torch.nn.ReLU(),
            torch.nn.MaxPool1d(kernel_size=2),
            torch.nn.Conv1d(in_channels  =128,out_channels =128,kernel_size =5,padding = 2),
            torch.nn.BatchNorm1d(128),
            torch.nn.ReLU(),
            torch.nn.MaxPool1d(kernel_size=2),
            torch.nn.AvgPool1d(kernel_size=125),
            torch.nn.Flatten(),
            torch.nn.Linear(128,2)
        )
    
    def forward(self, data):
        val = self.emb(data)
        val = val.permute(0, 2, 1)
        return self.model(val)

model = CNN()

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [21]:
# 5
from sklearn import metrics

minError = None

def trainEpoch(model):
    model.train()
    print(len(datasetTrain))
    for x, y in datasetTrain:
        optimizer.zero_grad()
        output = model(x)
        loss = loss_fn(output, y.type(torch.LongTensor))
        loss.backward()
        optimizer.step()


def evalModel(model):
    global minError
    model.eval()
    true_labels = []
    predicted_labels = []
    mean_loss = 0
    with torch.no_grad():  
        for (x, y) in datasetValidate:
            output = model(x)
            loss = loss_fn(output, y.type(torch.LongTensor))
            mean_loss += loss.item()
            true_labels.extend(y.tolist())
            predicted_labels.extend(output.max(1)[1].tolist())

    mean_loss /= len(datasetValidate)

    if minError != None: # aici am rezolvat exercitiul 6
        if minError > mean_loss:
            minError = mean_loss
            torch.save(model.state_dict(), "model.pt")
    else:
        minError = mean_loss
        torch.save(model.state_dict(), "model.pt")
    acc = metrics.accuracy_score(true_labels, predicted_labels)

    print("Accuracy:", acc)
    print("Mean loss:", mean_loss)


def trainModel(model, epochs):

    for _ in range(epochs):
        trainEpoch(model)
        evalModel(model)

In [24]:
trainModel(model,3)

625
Accuracy: 0.823
Mean loss: 0.419678065218503
625
Accuracy: 0.843
Mean loss: 0.36901676334157774
625
Accuracy: 0.8272
Mean loss: 0.39672634760035747


Am antrenat doar in 3 epoci deoarece dureaza, 6/8 minute si ceva sa antrenezi o epoca.

In [25]:
# 6
from sklearn.metrics import classification_report

model.load_state_dict(torch.load('model.pt'))
yPred = []
yGood = []
for x,y in datasetTest:
    y_pred = model(x)
    yGood.extend(y.tolist())
    yPred.extend(y_pred.max(1)[1].tolist())

print(classification_report(yGood,yPred)) 

              precision    recall  f1-score   support

           0       0.82      0.86      0.84      2490
           1       0.85      0.82      0.83      2510

    accuracy                           0.84      5000
   macro avg       0.84      0.84      0.84      5000
weighted avg       0.84      0.84      0.84      5000

